In [14]:
#import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Model, load_model # for creating a Neural Network Autoencoder model
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for adding layers to AE model
from tensorflow.keras.utils import plot_model #for plotting  model charts
from tensorflow.keras import models,layers,activations,losses,optimizers,metrics
from sklearn.preprocessing import RobustScaler, StandardScaler
from keras import regularizers
from sklearn.model_selection import train_test_split,cross_validate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, TransformerMixin
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor

In [15]:
#oad data
train_data = pd.read_pickle("EDA_train_median.pkl")
test_data = pd.read_pickle("EDA_test_median.pkl")

In [16]:
#separate target variable
y = train_data['windmill_generated_power(kW/h)']
train_data.drop(['windmill_generated_power(kW/h)'], axis=1, inplace = True)

In [17]:
# Create an instance of the scaler
scaler = StandardScaler()

# Fit the scaler to your data
scaler.fit(train_data)

# Transform the data using the scaler
scaled_data = scaler.transform(train_data)

In [18]:
#split dataset 

X_train, X_test, y_train, y_test = train_test_split(scaled_data, y, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((21051, 33), (21051,), (7018, 33), (7018,))

### GENERAL AUTOENCODER

In [19]:
n_inputs = X_train.shape[1] # number of input neurons = the number of features in X_train
n_bottleneck= 50

#--- Input Layer 
visible = Input(shape=(n_inputs,), name='Input-Layer') # Specify input shape

#--- Encoder Layer
e = Dense(units=200, name='Encoder-Layer1',activation=activations.relu, activity_regularizer=regularizers.l1(1e-3))(visible)
e = Dense(units=100, name='Encoder-Layer2',activation=activations.relu)(e)

#--- Bottleneck
bottleneck = Dense(units=n_bottleneck, name='Bottleneck-Layer')(e)

#--- Decoder Layer
d = Dense(units=100, name='Decoder-Layer1',activation=activations.relu)(bottleneck)
d = Dense(units=200, name='Decoder-Layer2',activation=activations.relu)(d)

#--- Output layer
output = Dense(units=n_inputs, activation='relu', name='Output-Layer')(d)

# Define autoencoder model
model = Model(inputs=visible, outputs=output, name='Autoencoder-Model')

# Compile autoencoder model
model.compile(optimizer='adam', loss='mae')

# Print model summary
print(model.summary())

# Train the autoencoder model
model.fit(X_train, X_train, epochs=30, batch_size=16, verbose=2, validation_data=(X_test, X_test))

Model: "Autoencoder-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input-Layer (InputLayer)    [(None, 33)]              0         
                                                                 
 Encoder-Layer1 (Dense)      (None, 200)               6800      
                                                                 
 Encoder-Layer2 (Dense)      (None, 100)               20100     
                                                                 
 Bottleneck-Layer (Dense)    (None, 50)                5050      
                                                                 
 Decoder-Layer1 (Dense)      (None, 100)               5100      
                                                                 
 Decoder-Layer2 (Dense)      (None, 200)               20200     
                                                                 
 Output-Layer (Dense)        (None, 33)          

C:\Users\Home\anaconda3\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


21051/21051 - 2s - loss: 0.5519 - val_loss: 0.5322 - 2s/epoch - 110us/sample
Epoch 2/30
21051/21051 - 2s - loss: 0.5261 - val_loss: 0.5241 - 2s/epoch - 102us/sample
Epoch 3/30
21051/21051 - 2s - loss: 0.5211 - val_loss: 0.5207 - 2s/epoch - 95us/sample
Epoch 4/30
21051/21051 - 2s - loss: 0.5191 - val_loss: 0.5187 - 2s/epoch - 102us/sample
Epoch 5/30
21051/21051 - 2s - loss: 0.5181 - val_loss: 0.5188 - 2s/epoch - 97us/sample
Epoch 6/30
21051/21051 - 2s - loss: 0.5176 - val_loss: 0.5183 - 2s/epoch - 90us/sample
Epoch 7/30
21051/21051 - 2s - loss: 0.5172 - val_loss: 0.5187 - 2s/epoch - 103us/sample
Epoch 8/30
21051/21051 - 2s - loss: 0.5171 - val_loss: 0.5176 - 2s/epoch - 100us/sample
Epoch 9/30
21051/21051 - 2s - loss: 0.5169 - val_loss: 0.5176 - 2s/epoch - 103us/sample
Epoch 10/30
21051/21051 - 2s - loss: 0.5167 - val_loss: 0.5179 - 2s/epoch - 99us/sample
Epoch 11/30
21051/21051 - 2s - loss: 0.5167 - val_loss: 0.5173 - 2s/epoch - 110us/sample
Epoch 12/30
21051/21051 - 2s - loss: 0.5166 -

### Decision Tree and General Autoencoder

In [20]:
X_encoded = model.predict(train_data)

# encode the train data
X_train_encode = model.predict(X_train)

# encode the test data
X_test_encode = model.predict(X_test)

# Train the decision tree model using the encoded features
dtr = DecisionTreeRegressor(max_depth = 30 , max_features = 'auto', min_samples_leaf = 30, min_samples_split = 20)
dtr.fit(X_train_encode, y_train)


#reshape test data
#X_test_encode_new = X_test_encode.reshape(-1, 1)
#y_test_new = y_test.to_numpy().reshape(-1, 1)

# prediction 
pred = dtr.predict(X_test_encode)

#metrics 
print('MAE:', mean_absolute_error(y_test,pred))
print('MSE:', mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, pred)))
print('Train score:',dtr.score(X_train_encode,y_train))
print('Test score:',dtr.score(X_test_encode,y_test))

C:\Users\Home\anaconda3\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


MAE: 1.157587516757572
MSE: 2.5321592688804175
RMSE: 1.5912759876528073
Train score: 0.7148905503621141
Test score: 0.6559349384472315


C:\Users\Home\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### Decision tree and Denoising Autoencoder

In [21]:
# Specify how much noise to add
level_of_noise = 0.00001

# Add random noise based on sampling from Gaussian distribution
X_train_noisy = X_train + level_of_noise * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + level_of_noise * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

# Enforce min-max boundaries so it does not go beyond [0,1] range
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)
# Print shapes
print("New shape of X_train: ", X_train.shape)
print("New shape of X_test: ", X_test.shape)

print("New shape of X_train_noisy: ", X_train_noisy.shape)
print("New shape of X_test_noisy: ", X_test_noisy.shape)


#--- Define Shapes
n_inputs = X_train_noisy.shape[1] # number of input neurons = the number of features in X_train
n_bottleneck= 50

#--- Input Layer 
visible = Input(shape=(n_inputs,), name='Input-Layer') # Specify input shape

#--- Encoder Layer
e = Dense(units=200, name='Encoder-Layer1',activation=activations.relu, activity_regularizer=regularizers.l1(1e-3))(visible)
e = Dense(units=100, name='Encoder-Layer2',activation=activations.relu)(e)

#--- Bottleneck
bottleneck = Dense(units=n_bottleneck, name='Bottleneck-Layer')(e)

#--- Decoder Layer
d = Dense(units=100, name='Decoder-Layer1',activation=activations.relu)(bottleneck)
d = Dense(units=200, name='Decoder-Layer2',activation=activations.relu)(d)

#--- Output layer
output = Dense(units=n_inputs, activation='relu', name='Output-Layer')(d)

# Define autoencoder model
model = Model(inputs=visible, outputs=output, name='Autoencoder-Model')

# Compile autoencoder model
model.compile(optimizer='adam', loss='mae')

# Print model summary
print(model.summary())

# Fit the autoencoder model to reconstruct input
history = model.fit(X_train_noisy, X_train_noisy, epochs=30, batch_size=32, verbose=1, validation_data=(X_test_noisy, X_test_noisy))

X_encoded_denoise = model.predict(train_data)

# encode the train data
X_train_encode_denoise = model.predict(X_train_noisy)

# encode the test data
X_test_encode_denoise = model.predict(X_test_noisy)

# Train the decision tree model using the encoded features
dtr_denoise = DecisionTreeRegressor(max_depth = 30 , max_features = 'auto', min_samples_leaf = 30, min_samples_split = 20)
dtr_denoise.fit(X_train_encode_denoise, y_train)

#test prediction
prediction_denoise = dtr_denoise.predict(X_test_encode_denoise)

# Calculate the MAE and MSE
mae = mean_absolute_error(y_test, prediction_denoise)
mse = mean_squared_error(y_test, prediction_denoise)
print('RMSE:', np.sqrt(mean_squared_error(y_test, prediction_denoise)))
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print('Train score:',dtr_denoise.score(X_train_encode_denoise,y_train))
print('Test score:',dtr_denoise.score(X_test_encode_denoise,y_test))

New shape of X_train:  (21051, 33)
New shape of X_test:  (7018, 33)
New shape of X_train_noisy:  (21051, 33)
New shape of X_test_noisy:  (7018, 33)
Model: "Autoencoder-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input-Layer (InputLayer)    [(None, 33)]              0         
                                                                 
 Encoder-Layer1 (Dense)      (None, 200)               6800      
                                                                 
 Encoder-Layer2 (Dense)      (None, 100)               20100     
                                                                 
 Bottleneck-Layer (Dense)    (None, 50)                5050      
                                                                 
 Decoder-Layer1 (Dense)      (None, 100)               5100      
                                                                 
 Decoder-Layer2 (Dense)      (Non

C:\Users\Home\anaconda3\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


21051/21051 [==============================] - 2s 92us/sample - loss: 0.1061 - val_loss: 0.0937
Epoch 2/30
21051/21051 [==============================] - 2s 72us/sample - loss: 0.0919 - val_loss: 0.0903
Epoch 3/30
21051/21051 [==============================] - 1s 70us/sample - loss: 0.0893 - val_loss: 0.0880
Epoch 4/30
21051/21051 [==============================] - 2s 75us/sample - loss: 0.0880 - val_loss: 0.0872
Epoch 5/30
21051/21051 [==============================] - 1s 70us/sample - loss: 0.0874 - val_loss: 0.0867
Epoch 6/30
21051/21051 [==============================] - 1s 67us/sample - loss: 0.0869 - val_loss: 0.0867
Epoch 7/30
21051/21051 [==============================] - 2s 76us/sample - loss: 0.0867 - val_loss: 0.0864
Epoch 8/30
21051/21051 [==============================] - 2s 76us/sample - loss: 0.0865 - val_loss: 0.0860
Epoch 9/30
21051/21051 [==============================] - 2s 77us/sample - loss: 0.0863 - val_loss: 0.0861
Epoch 10/30
21051/21051 [=======================

C:\Users\Home\anaconda3\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


RMSE: 1.7479312845218236
Mean Absolute Error: 1.279814523847353
Mean Squared Error: 3.0552637754101117
Train score: 0.6668375802482119
Test score: 0.5848564772897513


C:\Users\Home\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### Decision tree and Variational Autoencoder


In [22]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import backend as K
import tensorflow as tf


batch_size = 64
original_dim = (X_train.shape[1])
latent_dim = 50
intermediate_dim1 = 200
intermediate_dim2 = 100
epochs = 30
epsilon_std = 0.0001


# sampling from mean and sd in VAE
def sampling(args: tuple):
    # we grab the variables from the tuple
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

#########################
# input to our encoder
#########################
x = Input(shape=(original_dim,), name="input")

# intermediate layer
#h1 = Dense(intermediate_dim1, activation='tanh', name="encoding1")(x)
h1 = Dense(intermediate_dim1, activation="tanh", name="encoding1", activity_regularizer=regularizers.l1(10e-5))(x)
h = Dense(intermediate_dim2, activation='tanh', name="encoding")(h1)


# defining the mean of the latent space
z_mean = Dense(latent_dim, name="mean")(h)

# defining the log variance of the latent space
z_log_var = Dense(latent_dim, name="log-variance")(h)

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# defining the encoder as a keras model
encoder = Model(x, [z_mean, z_log_var, z], name="encoder")

# print out summary of what we just did
encoder.summary()


#########################
# Input to the decoder
#########################

input_decoder = Input(shape=(latent_dim,), name="decoder_input")

# taking the latent space to intermediate dimension
decoder_h1 = Dense(intermediate_dim2, activation='relu', name="decoder_h2")(input_decoder)
decoder_h = Dense(intermediate_dim1, activation='relu', name="decoder_h")(decoder_h1)

# getting the mean from the original dimension
x_decoded = Dense(original_dim, activation='tanh', name="flat_decoded")(decoder_h)

# defining the decoder as a keras model
decoder = Model(input_decoder, x_decoded, name="decoder")
decoder.summary()

##########################
# Variational Autoencoder
##########################

# grab the output. Recall, that we need to grab the 3rd element our sampling z
output_combined = decoder(encoder(x)[2])

# link the input and the overall output
vae = Model(x, output_combined)

# print out what the overall model looks like
vae.summary()

# Defina VAE Loss Function
def vae_loss(x: tf.Tensor, x_decoded_mean: tf.Tensor,z_log_var=z_log_var, z_mean=z_mean, original_dim=original_dim):
    xent_loss = original_dim * metrics.mae(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

vae.compile(optimizer='adam', loss=vae_loss, metrics=['mae'],experimental_run_tf_function=False)

history = vae.fit(X_train, X_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1).history


X_encoded = vae.predict(train_data)

# encode the train data
X_train_encode_vae = vae.predict(X_train)

# encode the test data
X_test_encode_vae = vae.predict(X_test)

# Train the decision tree model using the encoded features
dtr_vae = DecisionTreeRegressor(max_depth = 30 , max_features = 'auto', min_samples_leaf = 30, min_samples_split = 20)
dtr_vae.fit(X_train_encode_vae, y_train)

def get_error_term(v1, v2, _rmse=True):
    if _rmse:
        return np.sqrt(np.mean((v1 - v2) ** 2))
    #return MAE
    return np.mean(abs(v1 - v2))

X_train_pred = dtr_vae.predict(X_train_encode_vae)
mae_vector_train = get_error_term(X_train_pred, y_train, _rmse=False)
                   
X_pred = dtr_vae.predict(X_test_encode_vae)
mae_vector_test = get_error_term(X_pred, y_test, _rmse=False)
                   

#metrics 
print('MAE:', mean_absolute_error(y_test,X_pred))
print('MSE:', mean_squared_error(y_test, X_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, X_pred)))
print('Train score:',dtr_vae.score(X_train_encode_vae,y_train))
print('Test score:',dtr_vae.score(X_test_encode_vae,y_test))

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 33)]         0           []                               
                                                                                                  
 encoding1 (Dense)              (None, 200)          6800        ['input[0][0]']                  
                                                                                                  
 encoding (Dense)               (None, 100)          20100       ['encoding1[0][0]']              
                                                                                                  
 mean (Dense)                   (None, 50)           5050        ['encoding[0][0]']               
                                                                                            

C:\Users\Home\anaconda3\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


21051/21051 [==============================] - 2s 72us/sample - loss: 12.5454 - mae: 0.3501 - val_loss: 9.7701 - val_mae: 0.2755
Epoch 2/30
21051/21051 [==============================] - 1s 59us/sample - loss: 9.2958 - mae: 0.2649 - val_loss: 8.9579 - val_mae: 0.2573
Epoch 3/30
21051/21051 [==============================] - 1s 53us/sample - loss: 8.7484 - mae: 0.2524 - val_loss: 8.5683 - val_mae: 0.2477
Epoch 4/30
21051/21051 [==============================] - 1s 54us/sample - loss: 8.4212 - mae: 0.2436 - val_loss: 8.3083 - val_mae: 0.2404
Epoch 5/30
21051/21051 [==============================] - 1s 56us/sample - loss: 8.1483 - mae: 0.2352 - val_loss: 8.0122 - val_mae: 0.2309
Epoch 6/30
21051/21051 [==============================] - 1s 55us/sample - loss: 7.9034 - mae: 0.2279 - val_loss: 7.8259 - val_mae: 0.2261
Epoch 7/30
21051/21051 [==============================] - 1s 52us/sample - loss: 7.7265 - mae: 0.2235 - val_loss: 7.6456 - val_mae: 0.2217
Epoch 8/30
21051/21051 [=============

C:\Users\Home\anaconda3\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\Home\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


MAE: 0.7832773348978251
MSE: 1.4896315560310853
RMSE: 1.2205046317122623
Train score: 0.8563558604283825
Test score: 0.7975916525806077
